In [1]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import time

In [2]:
years = [2023,2022,2021,2020]
total_list = []
laliga_url = 'https://fbref.com/en/comps/12/La-Liga-Stats'

In [3]:
for year in years:
    data = requests.get(laliga_url)
    html = data.text #take html data
    bs = BS(html)
    time.sleep(5) #pause is crusial, because without it site with data would not let parsing
    total_table = bs.select('table.stats_table')[0]

    hyperlinks = [link.get('href') for link in total_table.find_all('a')] #getting link for every element that has 'a'- flag of hyperlink
    hyperlinks = [link for link in hyperlinks if '/squads/' in link]
    
    teams_url = [f'https://fbref.com{link}' for link in hyperlinks] #add url of every team in championship
    
    season_before = bs.select('a.prev')[0].get('href') #get link for next iteration (previous season)
    laliga_url = f'https://fbref.com{season_before}'
    
    for team in teams_url:
        team_name = team.split('/')[-1].replace('-Stats', '')
        data = requests.get(team)
        html = data.text
        matches = pd.read_html(html, match="Scores & Fixtures")[0]
        
        bs = BS(html)
        hyperlinks = [link.get('href') for link in bs.find_all('a')]
        hyperlinks = [link for link in hyperlinks if link and 'all_comps/shooting/' in link]
        
        data = requests.get(f'https://fbref.com{hyperlinks[0]}')
        html = data.text
        shooting = pd.read_html(html, match='Shooting')[0] #get data from other table with information about goals, shootings, etc.
        shooting.columns = shooting.columns.droplevel()
                            
        try:
            team_data = matches.merge(shooting[['Date','Sh', 'SoT','G/Sh','G/SoT', 'Dist','PK', 'FK', 'PKatt']], on='Date')
        except ValueError: #in case of absense of some values
            continue
            
        team_data = team_data[team_data['Comp'] == 'La Liga'] #leave only La Liga championship
        team_data['Season'] = f'{year-1}-{year}'
        team_data['Team'] = team_name
        
        total_list.append(team_data)
        time.sleep(5)

In [5]:
df = pd.concat(total_list)

In [6]:
df.to_csv('matches_laliga.csv')

In [7]:
df

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Sh,SoT,G/Sh,G/SoT,Dist,PK,FK,PKatt,Season,Team
0,2022-08-13,21:00,La Liga,Matchweek 1,Sat,Home,D,0,0,Rayo Vallecano,...,21.0,5.0,0.00,0.00,17.0,0.0,1.0,0.0,2022-2023,Barcelona
1,2022-08-21,22:00,La Liga,Matchweek 2,Sun,Away,W,4,1,Real Sociedad,...,15.0,7.0,0.27,0.57,14.6,0.0,0.0,0.0,2022-2023,Barcelona
2,2022-08-28,19:30,La Liga,Matchweek 3,Sun,Home,W,4,0,Valladolid,...,24.0,9.0,0.17,0.44,14.4,0.0,1.0,0.0,2022-2023,Barcelona
3,2022-09-03,21:00,La Liga,Matchweek 4,Sat,Away,W,3,0,Sevilla,...,18.0,5.0,0.17,0.60,16.0,0.0,2.0,0.0,2022-2023,Barcelona
5,2022-09-10,18:30,La Liga,Matchweek 5,Sat,Away,W,4,0,Cádiz,...,16.0,8.0,0.25,0.50,14.9,0.0,0.0,0.0,2022-2023,Barcelona
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50,2020-07-05,17:00,La Liga,Matchweek 34,Sun,Home,L,0,1,Leganés,...,7.0,2.0,0.00,0.00,18.9,0.0,1.0,0.0,2019-2020,Espanyol
51,2020-07-08,22:00,La Liga,Matchweek 35,Wed,Away,L,0,1,Barcelona,...,11.0,2.0,0.00,0.00,18.1,0.0,1.0,0.0,2019-2020,Espanyol
52,2020-07-12,14:00,La Liga,Matchweek 36,Sun,Home,L,0,2,Eibar,...,14.0,3.0,0.00,0.00,20.8,0.0,3.0,0.0,2019-2020,Espanyol
53,2020-07-16,21:00,La Liga,Matchweek 37,Thu,Away,L,0,1,Valencia,...,18.0,6.0,0.00,0.00,21.3,0.0,3.0,0.0,2019-2020,Espanyol
